In [18]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy


In [19]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 0. 1.] [1. 0. 0.] [0. 1. 0.]


In [57]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot1, datasigmaplot1=bayesianL.datiplot(C, dic, dics)
dataplot=np.log(dataplot1)
datasigmaplot=datasigmaplot1*np.sqrt(10)/dataplot1
sd=np.array([dataplot, datasigmaplot/max(datasigmaplot)])


numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [58]:
#Calcolo il fit bayesiano con glot e sd come test
N=20
mN, SN, y_infer, sy_infer, conta=\
bayesianL.bayesianpol(gplot, sd, 8, N, 0.0001, gplot,bethapar=10000, ifprint= True, ifwarning= True, nLbp=0)

parametri ottimali [-9.86060340e-02 -6.10536439e+01  3.46843303e+02 -8.79559539e+02
  5.95410870e+02  5.14782755e+02  1.96946575e+02  3.70643166e+01]
numero di armoniche cubiche 8
valore a kmin inferito -0.3808963201359431 bias -0.0986060340077897 dato a kmin -0.33587062547763863
determinante matrice delle armoniche cubiche ridotte: 6.455871081411576e-18


In [59]:
#Plotto il fit e i dati (gplot e sd)
%matplotlib widget
plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer),y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [68]:
#Calcolo il best fit su gplot e sd con 50 dati
N=20
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifbetha=True, ifprintbestfit= True, ifprintfinal= True, nLbf=0)
#Plotto il best fit e i dati (gplot e sd)



%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.')

numero di polinomi cubici fino al grado massimo  2 : 1
best alpha: 0.6829397019188246 deltalpha: -4.940825526489334e-11
best betha: 1.7404388543278626 deltbetha: 6.610045844013257e-12
logevidence: 488.94900079988565
contributi alla evidence:
dalla normalizzazione  -0.19067435373423483 501.4517158229199
dalla likelihood: -9.501235714668638
dalla derivata seconda della likelihood (log(det(A))): -2.8108049546313874
determinante della matrice delle armoniche cubiche ridotte 158.38014500713888 

numero di polinomi cubici fino al grado massimo  4 : 2
best alpha: 0.03129777663981996 deltalpha: -3.8106462429965404e-12
best betha: 6.732818335494417 deltbetha: 8.924239125462918e-11
logevidence: 756.1291482002283
contributi alla evidence:
dalla normalizzazione  -3.4642082178306213 772.0230221088937
dalla likelihood: -9.018240933314392
dalla derivata seconda della likelihood (log(det(A))): -3.411424757520261
determinante della matrice delle armoniche cubiche ridotte 19.52506441469954 

numero di p

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [61]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(range(2,(len(log_evidence_vP)+1)*2, 2), log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
#Calcolo il best fit includendo sempre piu' punti 
predbl=[]
spredbl=[]
Npointsbl=[]
for i in range(2):
    predb=[]
    spredb=[]
    Npointsb=[]
    for N in range(21, 96):
        
        mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
        bayesianL.bestfit(gplot, sd, N, gplot, ifbetha=True, ifprintbestfit=False, ifprintfinal=False, nLbf=i)
        Npointsb.append(np.linalg.norm(gplot[N]))
        if (N-21)%(70//5)==0: print(int((N-21)/70*100), '%')
        predb.append(mN[0])
        spredb.append(SN[0,0])
    predbl.append(predb)
    spredbl.append(spredb)
    Npointsbl.append(Npointsb)

0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %


In [67]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(1):
    #ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
    ax[0].errorbar(np.array(Npointsbl[i]), np.exp(np.array(predbl[i])), np.exp(np.array(predbl[i]))*np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
ax[0].fill_between(np.array(Npointsbl[0]), np.ones(len(Npointsbl[0]))*0.816-np.ones(len(Npointsbl[0]))*0.021, np.ones(len(Npointsbl[1]))*0.816+np.ones(len(Npointsbl[1]))*0.021, label='GK result', alpha=0.3)    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot1,datasigmaplot1*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 
ax[1].fill_between(np.linalg.norm(gplot, axis=1), np.ones(len(gplot))*0.816-np.ones(len(gplot))*0.021, np.ones(len(gplot))*0.816+np.ones(len(gplot))*0.021, label='GK result', alpha=0.3)    
ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')